### Criar Agente Analista Dados (versão 1)

In [1]:
import pandas as pd
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI

In [2]:
df_temp = pd.read_csv("https://datahub.io/core/global-temp/r/0.csv")
df_temp.head()

,Source,Year,Mean
0,gcag,1850,-0.4177
1,gcag,1851,-0.2333
2,gcag,1852,-0.2294
3,gcag,1853,-0.2704
4,gcag,1854,-0.2915


In [3]:
len(df_temp)

319

In [ ]:
chat = ChatOpenAI(model="gpt-3.5-turbo-0125")

agent_temp = create_pandas_dataframe_agent(
    chat,
    df_temp,
    verbose=True,
    agent_type="tool-calling"
)

In [ ]:
# 1- Qual temperatura média global do último ano
resposta = agent_temp.invoke({"input":"Qual a temperatura média global do último ano?"})
print(resposta)

In [5]:
# 2- QUal foi o ano com a maior temperatura média?
resposta = agent_temp.invoke({"input":"Qual foi o ano com maior temperatura média?"})
print(resposta)



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['Mean'] == df['Mean'].max()]['Year'].values[0]"}`


2024O ano com a maior temperatura média registrada no dataframe foi 2024.

> Finished chain.
{'input': 'Qual foi o ano com maior temperatura média?', 'output': 'O ano com a maior temperatura média registrada no dataframe foi 2024.'}


### Criar Agente Analista Dados (versão 2)

In [6]:
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits.sql.base import create_sql_agent
from langchain_openai import ChatOpenAI

In [7]:
db_sakila = SQLDatabase.from_uri("sqlite:///files/sakila_master.db")

chat = ChatOpenAI(model="gpt-3.5-turbo-0125")

agent_executor_sakila = create_sql_agent(
    chat,
    db=db_sakila,
    agent_type="tool-calling",
    verbose=True
)

In [8]:
response = agent_executor_sakila.invoke({"input": "Quais são os filmes mais alugados?"})
print(response)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


actor, address, category, city, country, customer, film, film_actor, film_category, film_text, inventory, language, payment, rental, staff, store
Invoking: `sql_db_schema` with `{'table_names': 'film'}`



CREATE TABLE film (
	film_id INTEGER NOT NULL, 
	title VARCHAR(255) NOT NULL, 
	description TEXT DEFAULT NULL, 
	release_year VARCHAR(4) DEFAULT NULL, 
	language_id SMALLINT NOT NULL, 
	original_language_id SMALLINT DEFAULT NULL, 
	rental_duration SMALLINT DEFAULT 3 NOT NULL, 
	rental_rate DECIMAL(4, 2) DEFAULT (4.99) NOT NULL, 
	length SMALLINT DEFAULT NULL, 
	replacement_cost DECIMAL(5, 2) DEFAULT (19.99) NOT NULL, 
	rating VARCHAR(10) DEFAULT 'G', 
	special_features VARCHAR(100) DEFAULT NULL, 
	last_update TIMESTAMP NOT NULL, 
	PRIMARY KEY (film_id), 
	CONSTRAINT fk_film_language FOREIGN KEY(language_id) REFERENCES language (language_id), 
	CONSTRAINT fk_film_language_original FOREIGN KEY(orig

c:\Dados\Guilherme\02-CURSOS\01-FAZENDO\UDEMY-Langchain\99999-agents\.venv\Lib\site-packages\langchain_community\utilities\sql_database.py:338: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "staff, store", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  metadata_table_names = [tbl.name for tbl in self._metadata.sorted_tables]
c:\Dados\Guilherme\02-CURSOS\01-FAZENDO\UDEMY-Langchain\99999-agents\.venv\Lib\site-packages\langchain_community\utilities\sql_database.py:350: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "staff, store", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  for tbl in self._metadata.sorted_table


Invoking: `sql_db_query` with `{'query': 'SELECT title, rental_duration, rental_rate, replacement_cost, rating FROM film ORDER BY rental_duration DESC LIMIT 10'}`


[('ADAPTATION HOLES', 7, 2.99, 18.99, 'NC-17'), ('ANONYMOUS HUMAN', 7, 0.99, 12.99, 'NC-17'), ('ARGONAUTS TOWN', 7, 0.99, 12.99, 'PG-13'), ('BIKINI BORROWERS', 7, 4.99, 26.99, 'NC-17'), ('BLACKOUT PRIVATE', 7, 2.99, 12.99, 'PG'), ('BLANKET BEVERLY', 7, 2.99, 21.99, 'G'), ('BOILED DARES', 7, 4.99, 13.99, 'PG'), ('BOONDOCK BALLROOM', 7, 0.99, 14.99, 'NC-17'), ('BORN SPINAL', 7, 4.99, 17.99, 'PG'), ('BORROWERS BEDAZZLED', 7, 0.99, 22.99, 'G')]Os filmes mais alugados com a duração de aluguel de 7 dias são:

1. ADAPTATION HOLES - Classificação NC-17, Preço de Aluguel $2.99, Custo de Reposição $18.99
2. ANONYMOUS HUMAN - Classificação NC-17, Preço de Aluguel $0.99, Custo de Reposição $12.99
3. ARGONAUTS TOWN - Classificação PG-13, Preço de Aluguel $0.99, Custo de Reposição $12.99
4. BIKINI BORROWERS - Classificação NC-17, Preço 

In [9]:
response = agent_executor_sakila.invoke({"input": "Quais atores atuaram em mais filmes?"})
print(response)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


actor, address, category, city, country, customer, film, film_actor, film_category, film_text, inventory, language, payment, rental, staff, store
Invoking: `sql_db_schema` with `{'table_names': 'actor, film_actor'}`



CREATE TABLE actor (
	actor_id NUMERIC NOT NULL, 
	first_name VARCHAR(45) NOT NULL, 
	last_name VARCHAR(45) NOT NULL, 
	last_update TIMESTAMP NOT NULL, 
	PRIMARY KEY (actor_id)
)

/*
3 rows from actor table:
actor_id	first_name	last_name	last_update
1.0000000000	PENELOPE	GUINESS	2020-12-23 07:12:29
2.0000000000	NICK	WAHLBERG	2020-12-23 07:12:29
3.0000000000	ED	CHASE	2020-12-23 07:12:29
*/


CREATE TABLE film_actor (
	actor_id INTEGER NOT NULL, 
	film_id INTEGER NOT NULL, 
	last_update TIMESTAMP NOT NULL, 
	PRIMARY KEY (actor_id, film_id), 
	CONSTRAINT fk_film_actor_actor FOREIGN KEY(actor_id) REFERENCES actor (actor_id) ON UPDATE CASCADE, 
	CONSTRAINT fk_film_actor_film FOREIGN KEY(f

c:\Dados\Guilherme\02-CURSOS\01-FAZENDO\UDEMY-Langchain\99999-agents\.venv\Lib\site-packages\langchain_community\utilities\sql_database.py:338: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "staff, store", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  metadata_table_names = [tbl.name for tbl in self._metadata.sorted_tables]
c:\Dados\Guilherme\02-CURSOS\01-FAZENDO\UDEMY-Langchain\99999-agents\.venv\Lib\site-packages\langchain_community\utilities\sql_database.py:350: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "staff, store", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  for tbl in self._metadata.sorted_table


Invoking: `sql_db_query` with `{'query': 'SELECT actor_id, COUNT(actor_id) AS movie_count FROM film_actor GROUP BY actor_id ORDER BY movie_count DESC LIMIT 10'}`


[(107, 42), (102, 41), (198, 40), (181, 39), (23, 37), (81, 36), (13, 35), (37, 35), (60, 35), (106, 35)]Os atores com os IDs 107, 102 e 198 atuaram em mais filmes, com 42, 41 e 40 filmes respectivamente.

> Finished chain.
{'input': 'Quais atores atuaram em mais filmes?', 'output': 'Os atores com os IDs 107, 102 e 198 atuaram em mais filmes, com 42, 41 e 40 filmes respectivamente.'}


In [10]:
response = agent_executor_sakila.invoke({"input": "Qual é o cliente com mais aluguéis?"})
print(response)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


actor, address, category, city, country, customer, film, film_actor, film_category, film_text, inventory, language, payment, rental, staff, store
Invoking: `sql_db_schema` with `{'table_names': 'customer, rental'}`



CREATE TABLE customer (
	customer_id INTEGER NOT NULL, 
	store_id INTEGER NOT NULL, 
	first_name VARCHAR(45) NOT NULL, 
	last_name VARCHAR(45) NOT NULL, 
	email VARCHAR(50) DEFAULT NULL, 
	address_id INTEGER NOT NULL, 
	active CHAR(1) DEFAULT 'Y' NOT NULL, 
	create_date TIMESTAMP NOT NULL, 
	last_update TIMESTAMP NOT NULL, 
	PRIMARY KEY (customer_id), 
	CONSTRAINT fk_customer_store FOREIGN KEY(store_id) REFERENCES store (store_id) ON UPDATE CASCADE, 
	CONSTRAINT fk_customer_address FOREIGN KEY(address_id) REFERENCES address (address_id) ON UPDATE CASCADE
)

/*
3 rows from customer table:
customer_id	store_id	first_name	last_name	email	address_id	active	create_date	last_update
1	1	MARY

c:\Dados\Guilherme\02-CURSOS\01-FAZENDO\UDEMY-Langchain\99999-agents\.venv\Lib\site-packages\langchain_community\utilities\sql_database.py:338: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "staff, store", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  metadata_table_names = [tbl.name for tbl in self._metadata.sorted_tables]
c:\Dados\Guilherme\02-CURSOS\01-FAZENDO\UDEMY-Langchain\99999-agents\.venv\Lib\site-packages\langchain_community\utilities\sql_database.py:350: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "staff, store", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  for tbl in self._metadata.sorted_table


Invoking: `sql_db_query` with `{'query': 'SELECT c.first_name, c.last_name, COUNT(r.rental_id) as total_rentals FROM customer c JOIN rental r ON c.customer_id = r.customer_id GROUP BY c.customer_id ORDER BY total_rentals DESC LIMIT 1'}`


[('ELEANOR', 'HUNT', 46)]O cliente com mais aluguéis é ELEANOR HUNT, com um total de 46 aluguéis.

> Finished chain.
{'input': 'Qual é o cliente com mais aluguéis?', 'output': 'O cliente com mais aluguéis é ELEANOR HUNT, com um total de 46 aluguéis.'}
